In [1]:
%cd ..

e:\projects\Chest X-Ray


In [2]:
import pandas as pd
data_map = pd.read_csv('data/data_map.csv')
data_map.head()

,imgs,clas,split,resized
0,data\chest_xray\chest_xray\test\NORMAL\IM-0001...,NORMAL,test,data/resized\test_NORMAL_IM-0001-0001.jpeg
1,data\chest_xray\chest_xray\test\NORMAL\IM-0003...,NORMAL,test,data/resized\test_NORMAL_IM-0003-0001.jpeg
2,data\chest_xray\chest_xray\test\NORMAL\IM-0005...,NORMAL,test,data/resized\test_NORMAL_IM-0005-0001.jpeg
3,data\chest_xray\chest_xray\test\NORMAL\IM-0006...,NORMAL,test,data/resized\test_NORMAL_IM-0006-0001.jpeg
4,data\chest_xray\chest_xray\test\NORMAL\IM-0007...,NORMAL,test,data/resized\test_NORMAL_IM-0007-0001.jpeg


In [28]:
from sklearn.preprocessing import LabelEncoder
data_map["clas_enc"]= LabelEncoder().fit_transform(data_map["clas"])

In [4]:
data_map

,imgs,clas,split,resized,clas_enc
0,data\chest_xray\chest_xray\test\NORMAL\IM-0001...,NORMAL,test,data/resized\test_NORMAL_IM-0001-0001.jpeg,0
1,data\chest_xray\chest_xray\test\NORMAL\IM-0003...,NORMAL,test,data/resized\test_NORMAL_IM-0003-0001.jpeg,0
2,data\chest_xray\chest_xray\test\NORMAL\IM-0005...,NORMAL,test,data/resized\test_NORMAL_IM-0005-0001.jpeg,0
3,data\chest_xray\chest_xray\test\NORMAL\IM-0006...,NORMAL,test,data/resized\test_NORMAL_IM-0006-0001.jpeg,0
4,data\chest_xray\chest_xray\test\NORMAL\IM-0007...,NORMAL,test,data/resized\test_NORMAL_IM-0007-0001.jpeg,0
...,...,...,...,...,...
1976,data\chest_xray\chest_xray\val\PNEUMONIA\perso...,PNEUMONIA,val,data/resized\val_PNEUMONIA_person1949_bacteria...,1
1977,data\chest_xray\chest_xray\val\PNEUMONIA\perso...,PNEUMONIA,val,data/resized\val_PNEUMONIA_person1950_bacteria...,1
1978,data\chest_xray\chest_xray\val\PNEUMONIA\perso...,PNEUMONIA,val,data/resized\val_PNEUMONIA_person1951_bacteria...,1
1979,data\chest_xray\chest_xray\val\PNEUMONIA\perso...,PNEUMONIA,val,data/resized\val_PNEUMONIA_person1952_bacteria...,1


# torch

In [11]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image

In [6]:
from torchvision import transforms, models

In [35]:


class XRayDataset(Dataset):
    def __init__(self, df_split, transform=None):
        """
        Args:
            df (pd.DataFrame): DataFrame يحتوي على مسارات الصور والفئات.
            split (str): نوع التقسيم ('train', 'val', 'test').
            transform (callable, optional): تحويلات سيتم تطبيقها على الصور. Defaults to None.
        """
        self.df_split = df_split
        self.transform = transform

    def __len__(self):
        """
        Returns:
            int: عدد الصور في مجموعة البيانات الفرعية المحددة.
        """
        return len(self.df_split)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): فهرس الصورة المطلوبة.

        Returns:
            tuple: (image, label) حيث image هو Tensor يمثل الصورة و label هو Tensor يمثل الفئة.
        """
        img_path = self.df_split.iloc[idx]['resized']
        clas_name = self.df_split.iloc[idx]['clas_enc']
        image = Image.open(img_path).convert('RGB')  # تحويل الصورة إلى تدرجات الرمادي
        image = self.transform(image)

        return image, clas_name

In [36]:
train_dataset = XRayDataset(data_map[data_map['split'] == 'train'],transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
]))
val_dataset = XRayDataset(data_map[data_map['split'] == 'val'],transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
]))

In [42]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# تحميل نموذج ResNet18 مسبقًا
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)


In [43]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Train

In [40]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss=nn.CrossEntropyLoss()

In [18]:
from tqdm import tqdm

In [32]:
train_dataset

In [44]:
for epoch in range(5):
    model.train()
    running_loss = 0
    for images, labels in tqdm(train_loader,total=len(train_loader)):
        # نقل البيانات إلى الجهاز المناسب (CPU أو GPU)
        images = images.to(device)
        labels = labels.to(device)
        
          # تحديث الأوزان
        optimizer.zero_grad()
        predictions = model(images)
        loss_value = loss(predictions, labels)
        loss_value.backward()
        optimizer.step()
        running_loss += loss_value.item()
    print(f'Epoch {epoch+1}/{5}, Loss: {running_loss/len(train_loader):.4f}')
    

100%|██████████| 42/42 [05:28<00:00,  7.82s/it]


Epoch 1/5, Loss: 8.8722


100%|██████████| 42/42 [03:57<00:00,  5.65s/it]


Epoch 2/5, Loss: 8.8747


100%|██████████| 42/42 [03:59<00:00,  5.71s/it]


Epoch 3/5, Loss: 8.8733


100%|██████████| 42/42 [03:57<00:00,  5.64s/it]


Epoch 4/5, Loss: 8.8751


100%|██████████| 42/42 [03:46<00:00,  5.40s/it]

Epoch 5/5, Loss: 8.8747


In [46]:
#save the model
torch.save(model.state_dict(), 'model.pth')